In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../")))

import random
import time

import numpy as np
import tensorflow as tf

tf.random.set_seed(3407)
np.random.seed(3407)
random.seed(3407)

from notebooks.helper_functions import (
    convert_bytes,
    convert_prefetchdataset_to_numpy_arrays,
    evaluate_prediction,
    evaluate_time_of_prediction,
    full_int_model_predict,
    get_f1_scores_of_bootstraping_partitions_full_int_q,
    get_f1_scores_of_non_overlapping_partitions_full_int_q,
    get_file_size,
    predict_and_print_full_results,
)

In [3]:
train_dataset = tf.keras.utils.audio_dataset_from_directory(
    "../../dataset/training",
    labels="inferred",
    sampling_rate=16000,
    batch_size=32,
    shuffle=True,
    seed=3407,
)
test_dataset = tf.keras.utils.audio_dataset_from_directory(
    "../../dataset/testing",
    labels="inferred",
    sampling_rate=16000,
    batch_size=32,
    shuffle=True,
    seed=3407,
)
val_dataset = tf.keras.utils.audio_dataset_from_directory(
    "../../dataset/validation",
    labels="inferred",
    sampling_rate=16000,
    batch_size=32,
    shuffle=True,
    seed=3407,
)

label_names = np.array(test_dataset.class_names)
print("Classes: ", label_names)

Found 11292 files belonging to 2 classes.


2024-09-22 17:13:52.651349: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-09-22 17:13:52.651828: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-09-22 17:13:52.717260: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
2024-09-22 17:13:52.720625: W tensorflow_io/core/kernels/audio_video_mp3_kernels.cc:271] libmp3lame.so.0 or lame functions are not available


Found 1393 files belonging to 2 classes.
Found 1380 files belonging to 2 classes.
Classes:  ['non_target' 'target']


In [4]:
x_train_np, y_train_np = convert_prefetchdataset_to_numpy_arrays(
    train_dataset, data_type="time-series"
)
x_val_np, y_val_np = convert_prefetchdataset_to_numpy_arrays(
    val_dataset, data_type="time-series"
)
x_test_np, y_test_np = convert_prefetchdataset_to_numpy_arrays(
    test_dataset, data_type="time-series"
)

In [8]:
BASE_MODEL_FILE_NAME = "../../models/time_series_models_from_notebooks/tiny_transformer/hpo/tiny_transformer_time_ser_final.keras"
baseline_model = tf.keras.models.load_model(BASE_MODEL_FILE_NAME)
baseline_model.summary()
convert_bytes(get_file_size(BASE_MODEL_FILE_NAME), "MB")

Model: "model_29"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_31 (InputLayer)       [(None, 48000, 1)]           0         []                            
                                                                                                  
 conv1d_88 (Conv1D)          (None, 47996, 32)            192       ['input_31[0][0]']            
                                                                                                  
 max_pooling1d_30 (MaxPooli  (None, 2999, 32)             0         ['conv1d_88[0][0]']           
 ng1D)                                                                                            
                                                                                                  
 dropout_116 (Dropout)       (None, 2999, 32)             0         ['max_pooling1d_30[0][0

### Convert to TF Lite format

In [7]:
converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp_8jglbfa/assets


INFO:tensorflow:Assets written to: /tmp/tmp_8jglbfa/assets
2024-09-22 16:19:42.057004: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-09-22 16:19:42.057057: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-09-22 16:19:42.058999: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp_8jglbfa
2024-09-22 16:19:42.061796: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-09-22 16:19:42.061832: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp_8jglbfa
2024-09-22 16:19:42.074136: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-09-22 16:19:42.077479: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-09-22 16:19:42.174319: I tensorflow/cc/saved_model/loader.cc:217] Running initializatio

In [11]:
print("Validation dataset:")
(
    y_pred_val,
    non_overlap_patritions_f1_scores_val,
    bootstrap_patritions_f1_scores_val,
) = predict_and_print_full_results(
    tflite_model, x_val_np, y_val_np, model_format="tf_lite"
)

print("\nTest dataset:")
(
    y_pred_val,
    non_overlap_patritions_f1_scores_val,
    bootstrap_patritions_f1_scores_val,
) = predict_and_print_full_results(
    tflite_model, x_test_np, y_test_np, model_format="tf_lite"
)

print("\nTime for Test dataset:")
evaluate_time_of_prediction(tflite_model, x_test_np, y_test_np, model_format="tf_lite")

TF_LITE_MODEL_FILE_NAME = "../../models/time_series_models_from_notebooks/tiny_transformer/quantized/tiny_transformer_time_series.tflite"
open(TF_LITE_MODEL_FILE_NAME, "wb").write(tflite_model)
print("\n")
print("Model file name: ", TF_LITE_MODEL_FILE_NAME)
convert_bytes(get_file_size(TF_LITE_MODEL_FILE_NAME), "MB")

Validation dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 96.52%
Recall: 95.34%
Precision: 94.54%
F1-score: 94.94%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9492733997046917

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9453512138412076

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 96.27%
Recall: 94.98%
Precision: 93.75%
F1-score: 94.36%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.942986273519618

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9483500786191553

Time for Test dataset:
Accuracy: 96.27%
Recall: 94.98%
Precision: 93.75%
F1-score: 94.36%

Time to make a prediction for a single data point
Mean: 0.006 seconds
Max: 0.011 seconds
M

In [12]:
convert_bytes(get_file_size(TF_LITE_MODEL_FILE_NAME), "KB")

File size: 43.238 Kilobytes


'43.238 Kilobytes'

### Dynamic range quantization

In [13]:
dynamic_range_quant_converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
dynamic_range_quant_converter.optimizations = [tf.lite.Optimize.DEFAULT]
dynamic_range_quant_model = dynamic_range_quant_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp9h9lakt2/assets


INFO:tensorflow:Assets written to: /tmp/tmp9h9lakt2/assets
2024-09-22 16:33:58.642641: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-09-22 16:33:58.642698: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-09-22 16:33:58.642965: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp9h9lakt2
2024-09-22 16:33:58.645661: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-09-22 16:33:58.645673: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp9h9lakt2
2024-09-22 16:33:58.653238: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-09-22 16:33:58.749103: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmp9h9lakt2
2024-09-22 16:33:58.776011: I tensorflow/cc/saved_model/loader.cc:316] SavedModel

In [14]:
print("Validation dataset:")
(
    y_pred_val,
    non_overlap_patritions_f1_scores_val,
    bootstrap_patritions_f1_scores_val,
) = predict_and_print_full_results(
    dynamic_range_quant_model, x_val_np, y_val_np, model_format="tf_lite"
)

print("\nTest dataset:")
(
    y_pred_test,
    non_overlap_patritions_f1_scores_test,
    bootstrap_patritions_f1_scores_test,
) = predict_and_print_full_results(
    dynamic_range_quant_model, x_test_np, y_test_np, model_format="tf_lite"
)

print("\nTime for Test dataset:")
evaluate_time_of_prediction(
    dynamic_range_quant_model, x_test_np, y_test_np, model_format="tf_lite"
)

DRQ_MODEL_FILE_NAME = "../../models/time_series_models_from_notebooks/tiny_transformer/quantized/tiny_transformer_time_series_drq.tflite"
open(DRQ_MODEL_FILE_NAME, "wb").write(dynamic_range_quant_model)
print("\n")
print("Model file name: ", DRQ_MODEL_FILE_NAME)
convert_bytes(get_file_size(DRQ_MODEL_FILE_NAME), "MB")

Validation dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 96.52%
Recall: 95.34%
Precision: 94.54%
F1-score: 94.94%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9492733997046917

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.947799869276201

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 96.27%
Recall: 94.98%
Precision: 93.75%
F1-score: 94.36%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.942986273519618

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9399935390662868

Time for Test dataset:
Accuracy: 96.27%
Recall: 94.98%
Precision: 93.75%
F1-score: 94.36%

Time to make a prediction for a single data point
Mean: 0.006 seconds
Max: 0.011 seconds
Mi

In [15]:
convert_bytes(get_file_size(DRQ_MODEL_FILE_NAME), "KB")

File size: 26.188 Kilobytes


'26.188 Kilobytes'

### Float16 quantization

In [16]:
float16_quant_converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
float16_quant_converter.optimizations = [tf.lite.Optimize.DEFAULT]
float16_quant_converter.target_spec.supported_types = [tf.float16]
float16_quant_model = float16_quant_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmppj41ovrm/assets


INFO:tensorflow:Assets written to: /tmp/tmppj41ovrm/assets
2024-09-22 16:36:32.456824: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-09-22 16:36:32.456877: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-09-22 16:36:32.457069: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmppj41ovrm
2024-09-22 16:36:32.462508: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-09-22 16:36:32.462540: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmppj41ovrm
2024-09-22 16:36:32.474600: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-09-22 16:36:32.558413: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmppj41ovrm
2024-09-22 16:36:32.588131: I tensorflow/cc/saved_model/loader.cc:316] SavedModel

In [19]:
print("Validation dataset:")
(
    y_pred_val,
    non_overlap_patritions_f1_scores_val,
    bootstrap_patritions_f1_scores_val,
) = predict_and_print_full_results(
    float16_quant_model, x_val_np, y_val_np, model_format="tf_lite"
)

print("\nTest dataset:")
(
    y_pred_test,
    non_overlap_patritions_f1_scores_test,
    bootstrap_patritions_f1_scores_test,
) = predict_and_print_full_results(
    float16_quant_model, x_test_np, y_test_np, model_format="tf_lite"
)

print("\nTime for Test dataset:")
evaluate_time_of_prediction(
    float16_quant_model, x_test_np, y_test_np, model_format="tf_lite"
)

FLOAT16_QUANT_MODEL_FILE_NAME = "../../models/time_series_models_from_notebooks/tiny_transformer/quantized/tiny_transformer_time_series_float16q.tflite"
open(FLOAT16_QUANT_MODEL_FILE_NAME, "wb").write(float16_quant_model)
print("\n")
print("Model file name: ", FLOAT16_QUANT_MODEL_FILE_NAME)
convert_bytes(get_file_size(FLOAT16_QUANT_MODEL_FILE_NAME), "MB")

Validation dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 96.52%
Recall: 95.34%
Precision: 94.54%
F1-score: 94.94%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9492733997046917

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9513203572414904

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 96.27%
Recall: 94.98%
Precision: 93.75%
F1-score: 94.36%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.942986273519618

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9366056979689855

Time for Test dataset:
Accuracy: 96.27%
Recall: 94.98%
Precision: 93.75%
F1-score: 94.36%

Time to make a prediction for a single data point
Mean: 0.005 seconds
Max: 0.013 seconds
M

In [20]:
convert_bytes(get_file_size(FLOAT16_QUANT_MODEL_FILE_NAME), "KB")

File size: 33.258 Kilobytes


'33.258 Kilobytes'

: 

### Full integer quantization

In [15]:
def representative_data_gen():
    for input_value in (
        tf.data.Dataset.from_tensor_slices(x_train_np).batch(1).take(10000)
    ):
        yield [input_value]


full_int_converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
full_int_converter.optimizations = [tf.lite.Optimize.DEFAULT]
full_int_converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
full_int_converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
full_int_converter.inference_input_type = tf.uint8
full_int_converter.inference_output_type = tf.uint8

full_int_model = full_int_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp6ixy8d2h/assets


INFO:tensorflow:Assets written to: /tmp/tmp6ixy8d2h/assets
/home/polina/newname/.venv/lib/python3.9/site-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2024-09-22 17:21:04.206901: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-09-22 17:21:04.206966: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-09-22 17:21:04.207223: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp6ixy8d2h
2024-09-22 17:21:04.210135: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-09-22 17:21:04.210176: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp6ixy8d2h
2024-09-22 17:21:04.220092: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-09-22 17:21

In [16]:
interpreter = tf.lite.Interpreter(model_content=full_int_model)
input_type = interpreter.get_input_details()[0]["dtype"]
print("input: ", input_type)
output_type = interpreter.get_output_details()[0]["dtype"]
print("output: ", output_type)

input:  <class 'numpy.uint8'>
output:  <class 'numpy.uint8'>


In [17]:
import pathlib

tflite_models_dir = pathlib.Path(
    "../../models/time_series_models_from_notebooks/tiny_transformer/quantized/"
)
tflite_models_dir.mkdir(exist_ok=True, parents=True)

# Save the quantized model:
tflite_model_full_int_q_file = (
    tflite_models_dir / "tiny_transformer_time_series_full_int_q.tflite"
)
tflite_model_full_int_q_file.write_bytes(full_int_model)

30056

In [18]:
print("Validation dataset:")
predictions = full_int_model_predict(tflite_model_full_int_q_file, x_val_np)
evaluate_prediction(y_val_np, predictions)

# print("\nDevide dataset into 10 non-overlapping patritions and get their mean F1-score")
# non_overlap_patritions_f1_scores = get_f1_scores_of_non_overlapping_partitions_full_int_q(tflite_model_full_int_q_file, x_val_np, y_val_np)
# print("Non-overlap mean F1-score: ", np.mean(non_overlap_patritions_f1_scores))

# print("\nGet 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score")
# bootstrap_patritions_f1_scores = get_f1_scores_of_bootstraping_partitions_full_int_q(tflite_model_full_int_q_file, x_val_np, y_val_np)
# print("Bootstrap mean F1-score: ", np.mean(bootstrap_patritions_f1_scores))


print("\nTest dataset:")
predictions = full_int_model_predict(tflite_model_full_int_q_file, x_test_np)
evaluate_prediction(y_test_np, predictions)

# print("\nDevide dataset into 10 non-overlapping patritions and get their mean F1-score")
# non_overlap_patritions_f1_scores = get_f1_scores_of_non_overlapping_partitions_full_int_q(tflite_model_full_int_q_file, x_test_np, y_test_np)
# print("Non-overlap mean F1-score: ", np.mean(non_overlap_patritions_f1_scores))

# print("\nGet 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score")
# bootstrap_patritions_f1_scores = get_f1_scores_of_bootstraping_partitions_full_int_q(tflite_model_full_int_q_file, x_test_np, y_test_np)
# print("Bootstrap mean F1-score: ", np.mean(bootstrap_patritions_f1_scores))

print("\nTime for Test dataset:")
time_data = []
for data_point in x_test_np:
    start_time = time.time()
    predictions = full_int_model_predict(tflite_model_full_int_q_file, [data_point])
    elapsed_time = time.time() - start_time
    time_data.append(elapsed_time)
print("\nTime to make a prediction for a single data point")
print(f"Mean: {round(np.mean(time_data), 3)} seconds")
print(f"Max: {round(np.max(time_data), 3)} seconds")
print(f"Min: {round(np.min(time_data), 3)} seconds")

convert_bytes(get_file_size(tflite_model_full_int_q_file), "KB")

Validation dataset:
Accuracy: 68.12%
Recall: 7.42%
Precision: 92.11%
F1-score: 13.73%

Test dataset:
Accuracy: 70.14%
Recall: 9.39%
Precision: 97.73%
F1-score: 17.13%

Time for Test dataset:

Time to make a prediction for a single data point
Mean: 0.008 seconds
Max: 0.032 seconds
Min: 0.006 seconds
File size: 29.352 Kilobytes


'29.352 Kilobytes'

We need to adjust decision threshold